In [1]:
# set working directory :
import os
pwd = os.getcwd() + "/../"
os.chdir(pwd)


In [2]:
import json
from tickapp.transformers.receipt_transformer import ReceiptTransformer
from tickapp.clients.database_client import DatabaseClient

In [3]:
# JSON exemple (celui de ton ticket Aeschbach)
CLAUDE_JSON = {
  "magasin": {
    "nom": "Aeschbach",
    "adresse": "Avenue Louis-Casaï 27",
    "code_postal": "1211",
    "ville": "Genève",
    "pays": "CH",
    "telephone": "022 979 09 00"
  },
  "transaction": {
    "category_id": 2,
    "numero_ticket": "1755500",
    "date": "2023-11-21",
    "heure": "16:39:00",
    "mode_paiement": "carte"
  },
  "devise": "CHF",
  "articles": [
    {
      "nom": "Textile",
      "reference": "99950074-XL",
      "marque": "",
      "quantite": 1,
      "prix_unitaire": 97.30,
      "prix_total": 97.30,
      "categorie": "Vêtements et chaussures",
      "sous_categorie": "Vêtements femme",
      "tva": ""
    },
    {
      "nom": "Textile",
      "reference": "99771399M-M",
      "marque": "",
      "quantite": 1,
      "prix_unitaire": 143.20,
      "prix_total": 143.20,
      "categorie": "Vêtements et chaussures",
      "sous_categorie": "Vêtements femme",
      "tva": ""
    }
  ],
  "total": 240.50
}

In [4]:
print("=" * 60)
print("📝 TEST D'INSERTION D'UN TICKET")
print("=" * 60)

# ====================================================================
# 1. TRANSFORMER LE JSON CLAUDE EN OBJETS PYTHON
# ====================================================================
print("\n1️⃣  Transformation du JSON Claude en objets Python...")

receipt_data = ReceiptTransformer.transform_claude_json(
    claude_json=CLAUDE_JSON,
    message_id=None  # Pas de message Signal pour ce test
)

print(f"   ✅ Magasin: {receipt_data.store.store_name}")
print(f"   ✅ Transaction: {receipt_data.transaction.transaction_date}")
print(f"   ✅ Articles: {len(receipt_data.items)}")

📝 TEST D'INSERTION D'UN TICKET

1️⃣  Transformation du JSON Claude en objets Python...
   ✅ Magasin: Aeschbach
   ✅ Transaction: 2023-11-21
   ✅ Articles: 2


In [5]:
# ====================================================================
# 2. INSÉRER DANS LA BASE DE DONNÉES
# ====================================================================
print("\n2️⃣  Insertion dans la base de données...")

db_client = DatabaseClient(
    host="localhost",
    port=5434,
    database="receipt_processing",
    user="receipt_user",
    password="SuperSecretPassword123!"
)

transaction_id = db_client.insert_receipt(
    receipt_data=receipt_data,
    message_id=None,
    attachment_ids=None
)


2️⃣  Insertion dans la base de données...
✅ Ticket inséré : transaction_id=2, 2 articles


In [6]:
# ====================================================================
# 3. VÉRIFICATION
# ====================================================================
print("\n3️⃣  Vérification de l'insertion...")

# Vérifier que transaction_id est défini
if 'transaction_id' not in locals() and 'transaction_id' not in globals():
    print("⚠️  transaction_id n'est pas défini. Exécutez d'abord la Cell 4 !")
    # Optionnel : récupérer la dernière transaction
    import psycopg2
    conn = psycopg2.connect(
        host="localhost",
        port=5434,
        database="receipt_processing",
        user="receipt_user",
        password="SuperSecretPassword123!"
    )
    cursor = conn.cursor()
    cursor.execute("SELECT MAX(transaction_id) FROM transaction")
    transaction_id = cursor.fetchone()[0]
    print(f"   → Utilisation de la dernière transaction: {transaction_id}")
    cursor.close()
    conn.close()

import psycopg2
conn = psycopg2.connect(
    host="localhost",
    port=5434,
    database="receipt_processing",
    user="receipt_user",
    password="SuperSecretPassword123!"
)
cursor = conn.cursor()

# Récupérer la transaction
cursor.execute("""
    SELECT 
        t.transaction_id,
        t.transaction_date,
        t.total,
        t.currency,
        s.store_name,
        s.city,
        COUNT(i.item_id) as nb_items
    FROM transaction t
    JOIN store s ON t.store_id = s.store_id
    LEFT JOIN transaction_item_mapping tim ON t.transaction_id = tim.transaction_id
    LEFT JOIN item i ON tim.item_id = i.item_id
    WHERE t.transaction_id = %s
    GROUP BY t.transaction_id, s.store_id
""", (transaction_id,))

result = cursor.fetchone()

print("\n" + "=" * 60)
print("🎉 RÉSULTAT")
print("=" * 60)
print(f"\n📊 Transaction ID: {result[0]}")
print(f"📅 Date: {result[1]}")
print(f"🏪 Magasin: {result[4]} ({result[5]})")
print(f"💰 Total: {result[2]} {result[3]}")
print(f"📦 Nombre d'articles: {result[6]}")


3️⃣  Vérification de l'insertion...

🎉 RÉSULTAT

📊 Transaction ID: 2
📅 Date: 2023-11-21
🏪 Magasin: Aeschbach (Genève)
💰 Total: 240.50 CHF
📦 Nombre d'articles: 2


In [7]:
# Lister les articles
# Vérifier que transaction_id est défini
if 'transaction_id' not in locals() and 'transaction_id' not in globals():
    print("⚠️  transaction_id n'est pas défini. Exécutez d'abord la Cell 4 !")
    # Optionnel : récupérer la dernière transaction
    import psycopg2
    conn = psycopg2.connect(
        host="localhost",
        port=5434,
        database="receipt_processing",
        user="receipt_user",
        password="SuperSecretPassword123!"
    )
    cursor = conn.cursor()
    cursor.execute("SELECT MAX(transaction_id) FROM transaction")
    transaction_id = cursor.fetchone()[0]
    print(f"   → Utilisation de la dernière transaction: {transaction_id}")
    cursor.close()
    conn.close()

cursor.execute("""
    SELECT 
        i.product_name,
        c.category_main,
        c.category_sub,
        i.total_price
    FROM transaction_item_mapping tim
    JOIN item i ON tim.item_id = i.item_id
    JOIN item_category c ON i.category_id = c.category_id
    WHERE tim.transaction_id = %s
    ORDER BY i.line_number
""", (transaction_id,))

print(f"\n📋 Détail des articles:")
for item in cursor.fetchall():
    print(f"   • {item[0]}")
    print(f"     Catégorie: {item[1]} > {item[2]}")
    print(f"     Prix: {item[3]} CHF")

cursor.close()
conn.close()

print("\n" + "=" * 60)
print("✅ TEST TERMINÉ AVEC SUCCÈS !")
print("=" * 60)
print(f"\n💡 Pour voir dans pgAdmin: http://localhost:5434")
print(f"   Transaction ID: {transaction_id}")



📋 Détail des articles:
   • Textile
     Catégorie: Vêtements et chaussures > Vêtements femme
     Prix: 97.30 CHF
   • Textile
     Catégorie: Vêtements et chaussures > Vêtements femme
     Prix: 143.20 CHF

✅ TEST TERMINÉ AVEC SUCCÈS !

💡 Pour voir dans pgAdmin: http://localhost:5434
   Transaction ID: 2
